# 0 Download And Prepare Taxi Trip Data

We will start by downloading the data from the New York City Taxi and Limousine Commissions's website. For the purposes of our analysis, we have picked the yellow taxi trip data for July 2015. We specifically chose a 2015 data set because it contains individual dropoffs and pickups at the latitude/longitude level, which will come in handy in Step 3. Beginning in 2016, data is no longer provided at the latitude/longitude level.

#### Download links
[Yellow taxi trip data - July 2015](https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2015-07.csv) *copy this to taxiJul.csv

[Taxi Zone lookup file (csv)](https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv)

[Taxi Zone shape file](https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip)

[Data Dictionary](http://www.nyc.gov/html/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf)


In [1]:
%%time
#We will load the first 10 rows of the csv file into sample dataframe. We will use this sample to examine the columns
import pandas as pd
sample = pd.read_csv('taxiJul.csv',nrows=10)

Wall time: 1.37 s


In [2]:
#Show the columns in the data
sample.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RatecodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')

# QGIS Spatial Joins

The next step is to use QGIS to perform spatial joins to enrich the data. Use QGIS to do the following operations:

1. Join the July trip data file (CSV) with the Taxi Zone shape file by doing a spatial join based on *pickup latitude and longitude*. This gives you the pickup_zone and pickup_borough fields (among others) which are useful for analysis
1. Join the July trip data file (CSV) with the Taxi Zone shape file by doing a spatial join based on *dropoff latitude and longitude*. This gives you the dropoff_zone and dropoff_borough fields (among others) which are useful for analysis

#### How to do a spatial join

A spatial join is a powerful technique, where you take a CSV dataset that has latitudes and longitudes, JOIN it to another “shapefile” that contains geographical attributes (eg. Zip codes, city names, etc), and ENRICH the original dataset with those additional attributes.

QGIS (available as a free download) is used to accomplish this.

First, load the taxi trip data (CSV) by choosing Layer -> Add Layer -> Add Delimited Text Layer

<img alt="QGIS Add CSV Layer" src="QGISAddCSVLayer.png" width="600">

Next, load the taxi zone shapefile by choosing Layer -> Add Layer -> Add Vector Layer

<img alt="QGIS Add Vector Layer" src="QGISAddVectorLayer.png" width="300">

Now save this using the WGS84 coordinate reference system. In order to successfully join the information, the two vectors must use the same “Coordinate Reference System”. Use EPSG:4326, WGS84 to represent latitudes and longitudes. In this example, the taxi trip data was already tagged with latitudes and longitudes, so simply select WGS84 as the CRS. For the taxi zones, Select the layer, Choose SAVE AS and save it using the WGS84 CRS.

<img alt="QGIS Save Vector As WGS84" src="QGISSaveVectorAsWGS84.PNG" width="300">

You then go to Vector -> Data Management Tools -> Join attributes by location to perform a spatial join. Select “within” as the join operator.

<img alt="QGIS Spatial Join" src="QGISSpatialJoin.png" width="600">

QGIS is a powerful data visualization tool, and has a lot of options for handling formatting, labels, colors, etc.


#### Store the output
Store the result as a CSV file named 'taxiJul_enh2.csv', which we will import into a database in the next step

# Load Into SQLite Database

In this step, we load the data into a relational database (SQLite). The database comes in handy for processing large amounts of data where the entire data set may not fit into memory.

You can use the power of the SQL query language to define Views to prepare the data. It also gives you the flexibility to perform set operations (if desired) and filter using the WHERE clause to pull subsets for in-memory processing

In [3]:
%%time
#Create the table
commands=""".mode csv
.headers on
DROP TABLE taxiJul;
CREATE TABLE taxiJul(
  "VendorID" TEXT,
  "tpep_pickup_datetime" TEXT,
  "tpep_dropoff_datetime" TEXT,
  "passenger_count" NUMERIC,
  "trip_distance" NUMERIC,
  "pickup_longitude" TEXT,
  "pickup_latitude" TEXT,
  "RatecodeID" TEXT,
  "store_and_fwd_flag" TEXT,
  "dropoff_longitude" TEXT,
  "dropoff_latitude" TEXT,
  "payment_type" TEXT,
  "fare_amount" NUMERIC,
  "extra" NUMERIC,
  "mta_tax" NUMERIC,
  "tip_amount" NUMERIC,
  "tolls_amount" NUMERIC,
  "improvement_surcharge" NUMERIC,
  "total_amount" NUMERIC,
  "pickup_OBJECTID" TEXT,
  "pickup_Shape_Leng" NUMERIC,
  "pickup_Shape_Area" NUMERIC,
  "pickup_zone" TEXT,
  "pickup_LocationID" TEXT,
  "pickup_borough" TEXT,
  "dropoff_OBJECTID" TEXT,
  "dropoff_Shape_Leng" NUMERIC,
  "dropoff_Shape_Area" NUMERIC,
  "dropoff_zone" TEXT,
  "dropoff_Location" TEXT,
  "dropoff_borough" TEXT
);
.import taxiJul_enh2.csv taxiJul
"""
print(commands)
with open('CreateTableTaxiJul.sql', 'w') as f:
  f.write(commands)


.mode csv
.headers on
DROP TABLE taxiJul;
CREATE TABLE taxiJul(
  "VendorID" TEXT,
  "tpep_pickup_datetime" TEXT,
  "tpep_dropoff_datetime" TEXT,
  "passenger_count" NUMERIC,
  "trip_distance" NUMERIC,
  "pickup_longitude" TEXT,
  "pickup_latitude" TEXT,
  "RatecodeID" TEXT,
  "store_and_fwd_flag" TEXT,
  "dropoff_longitude" TEXT,
  "dropoff_latitude" TEXT,
  "payment_type" TEXT,
  "fare_amount" NUMERIC,
  "extra" NUMERIC,
  "mta_tax" NUMERIC,
  "tip_amount" NUMERIC,
  "tolls_amount" NUMERIC,
  "improvement_surcharge" NUMERIC,
  "total_amount" NUMERIC,
  "pickup_OBJECTID" TEXT,
  "pickup_Shape_Leng" NUMERIC,
  "pickup_Shape_Area" NUMERIC,
  "pickup_zone" TEXT,
  "pickup_LocationID" TEXT,
  "pickup_borough" TEXT,
  "dropoff_OBJECTID" TEXT,
  "dropoff_Shape_Leng" NUMERIC,
  "dropoff_Shape_Area" NUMERIC,
  "dropoff_zone" TEXT,
  "dropoff_Location" TEXT,
  "dropoff_borough" TEXT
);
.import taxiJul_enh2.csv taxiJul

Wall time: 2.01 ms


In [4]:
%%time
!sqlite3 taxiJul.db <CreateTableTaxiJul.sql

Wall time: 3min 29s


# Prepare DateTime Fields For Analysis

Next, we further prepare the data for analysis by splitting the composite DateTime fields into year, month, day, hour, minute, second and weekday. This will come in handy later when we analyze by those slices

In [5]:
%%time
#Define the view
commands=""".mode csv
.headers on
DROP VIEW taxiJulEnrich; 
CREATE VIEW taxiJulEnrich AS SELECT *,
1 AS `count`,
strftime('%Y',`tpep_pickup_datetime`) AS `pu_year`, 
strftime('%m',`tpep_pickup_datetime`) AS `pu_month`, 
strftime('%d',`tpep_pickup_datetime`) AS `pu_day`, 
strftime('%H',`tpep_pickup_datetime`) AS `pu_hour`, 
strftime('%M',`tpep_pickup_datetime`) AS `pu_minute`, 
strftime('%S',`tpep_pickup_datetime`) AS `pu_second`, 
strftime('%w',`tpep_pickup_datetime`) AS `pu_weekday`,
strftime('%Y',`tpep_dropoff_datetime`) AS `do_year`, 
strftime('%m',`tpep_dropoff_datetime`) AS `do_month`, 
strftime('%d',`tpep_dropoff_datetime`) AS `do_day`, 
strftime('%H',`tpep_dropoff_datetime`) AS `do_hour`, 
strftime('%M',`tpep_dropoff_datetime`) AS `do_minute`, 
strftime('%S',`tpep_dropoff_datetime`) AS `do_second`, 
strftime('%w',`tpep_dropoff_datetime`) AS `do_weekday`,
`pickup_latitude`||','||`pickup_longitude` AS `pu_latlong`
FROM taxiJul; 
SELECT * from taxiJulEnrich LIMIT 10;"""
print(commands)
with open('CreateViewTaxiJulEnrich.sql', 'w') as f:
  f.write(commands)


.mode csv
.headers on
DROP VIEW taxiJulEnrich; 
CREATE VIEW taxiJulEnrich AS SELECT *,
1 AS `count`,
strftime('%Y',`tpep_pickup_datetime`) AS `pu_year`, 
strftime('%m',`tpep_pickup_datetime`) AS `pu_month`, 
strftime('%d',`tpep_pickup_datetime`) AS `pu_day`, 
strftime('%H',`tpep_pickup_datetime`) AS `pu_hour`, 
strftime('%M',`tpep_pickup_datetime`) AS `pu_minute`, 
strftime('%S',`tpep_pickup_datetime`) AS `pu_second`, 
strftime('%w',`tpep_pickup_datetime`) AS `pu_weekday`,
strftime('%Y',`tpep_dropoff_datetime`) AS `do_year`, 
strftime('%m',`tpep_dropoff_datetime`) AS `do_month`, 
strftime('%d',`tpep_dropoff_datetime`) AS `do_day`, 
strftime('%H',`tpep_dropoff_datetime`) AS `do_hour`, 
strftime('%M',`tpep_dropoff_datetime`) AS `do_minute`, 
strftime('%S',`tpep_dropoff_datetime`) AS `do_second`, 
strftime('%w',`tpep_dropoff_datetime`) AS `do_weekday`,
`pickup_latitude`||','||`pickup_longitude` AS `pu_latlong`
FROM taxiJul; 
SELECT * from taxiJulEnrich LIMIT 10;
Wall time: 1 ms


In [6]:
%%time
!sqlite3 taxiJul.db <CreateViewTaxiJulEnrich.sql

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup_OBJECTID,pickup_Shape_Leng,pickup_Shape_Area,pickup_zone,pickup_LocationID,pickup_borough,dropoff_OBJECTID,dropoff_Shape_Leng,dropoff_Shape_Area,dropoff_zone,dropoff_Location,dropoff_borough,count,pu_year,pu_month,pu_day,pu_hour,pu_minute,pu_second,pu_weekday,do_year,do_month,do_day,do_hour,do_minute,do_second,do_weekday,pu_latlong
VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,OBJECTID_1,Shape_Le_1,Shape_Ar_1,zone_1,Location

In [7]:
%%time
!sqlite3 taxiJul.db "DROP TABLE `zones`;"
!ECHO .import taxi+_zone_lookup.csv zones>script.txt & sqlite3 -csv -header taxiJul.db <script.txt
!sqlite3 -header taxiJul.db "SELECT * FROM `zones`;"

LocationID|Borough|Zone|service_zone
1|EWR|Newark Airport|EWR
2|Queens|Jamaica Bay|Boro Zone
3|Bronx|Allerton/Pelham Gardens|Boro Zone
4|Manhattan|Alphabet City|Yellow Zone
5|Staten Island|Arden Heights|Boro Zone
6|Staten Island|Arrochar/Fort Wadsworth|Boro Zone
7|Queens|Astoria|Boro Zone
8|Queens|Astoria Park|Boro Zone
9|Queens|Auburndale|Boro Zone
10|Queens|Baisley Park|Boro Zone
11|Brooklyn|Bath Beach|Boro Zone
12|Manhattan|Battery Park|Yellow Zone
13|Manhattan|Battery Park City|Yellow Zone
14|Brooklyn|Bay Ridge|Boro Zone
15|Queens|Bay Terrace/Fort Totten|Boro Zone
16|Queens|Bayside|Boro Zone
17|Brooklyn|Bedford|Boro Zone
18|Bronx|Bedford Park|Boro Zone
19|Queens|Bellerose|Boro Zone
20|Bronx|Belmont|Boro Zone
21|Brooklyn|Bensonhurst East|Boro Zone
22|Brooklyn|Bensonhurst West|Boro Zone
23|Staten Island|Bloomfield/Emerson Hill|Boro Zone
24|Manhattan|Bloomingdale|Yellow Zone
25|Brooklyn|Boerum Hill|Boro Zone
26|Brooklyn|Borough Park|Boro Zone
27|Queens|Breezy Point/Fort Tilden/Riis Be